In [300]:
import pandas as pd
import media_mapper as mm
import numpy as np
from sklearn.decomposition import NMF
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.metrics.pairwise import linear_kernel
import tokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
import json

In [17]:
def retrieve_and_merge_tweet_data():
    '''
    INPUT: None. Retrieves  1) twitter geo data from SQL 2) tokenized tweet text. 
    OUTPUT: A merged dataframe containing tweet text and geo shape files.'''
    #get SF Data From SQL
    df = mm.pipeline.retrieve_sql_tweets('tweets_with_geoV6')
    #get text data from picke
    dftxt = pd.read_csv('data/intermediate_data/json_tweets_in_df_twitokend.csv')
    df = df.set_index('id')
    dftxt = dftxt.set_index('id')
    dfall = df.join(dftxt).reset_index()
    dfall.drop('Unnamed: 0', 1, inplace = True)
    return dfall

In [353]:
df = retrieve_and_merge_tweet_data()

#1. Cluster Tweets Based on Geograph

In [354]:
df.head()

id   timestamp_ms          geoid10  \
0  621410300290592768  1436990735491  060750615003006   
1  625416235082194944  1437945824772  060750134001000   
2  625816517381332992  1438041259509  060759804011003   
3  625112776483188736  1437873474604  060750101001001   
4  625345896507666432  1437929054749  060750604001007   

                                                text  
0  [u"I'm", u'at', u'@Google', u'San', u'Francisc...  
1  [u'Clear', u'sky', u'view', u'of', u'SF', u'@'...  
2                        [u'http://t.co/aGnxhinfIM']  
3  [u'a', u"gentleman's", u'quarters', u'at', u'#...  
4  [u'\u0421\u043f\u043e\u0439', u',', u'\u043f\u...

In [355]:
#get a grouped sum of the words
dftxt = df.groupby(['geoid10'])['text'].apply(lambda x: ','.join(x)).reset_index()

#make a list of every id for every text in case it is every useful...
dfids = df.groupby(['geoid10'])['id'].apply(lambda x: x.tolist())
#merge these two dataframes together


In [362]:
dftxt15clust['geoid10'] = dftxt['geoid10']
dftxt10clust['geoid10'] = dftxt['geoid10']

In [211]:
geo_docs = dftxt.text.tolist()

#Tokenize the Text

In [105]:
def create_tfidf(list_of_documents, max_feat):
    ''' transforms a list of documents into a matrix of 
        tf-idf (term frequency - inverse document frequency) 
        INPUT: a list of documents (tweets)
        OUTPUT: a TFIDF object.
    Returns the document term matrix, feature names, and the trained model'''
    #initialize a TfidfVectorizer
    vectorizer = TfidfVectorizer(max_features = max_feat, stop_words='english', strip_accents='unicode')
    #Transform the words into a TFIDF matrix
    tfidf_vectorized = vectorizer.fit_transform(geo_docs)
    document_term_mat = tfidf_vectorized.todense()
    #so i have a matrix with 144,287 features, and only 3,794 documents.
    tfidf_sf_geo.shape
    feature_names = vectorizer.get_feature_names()
    return document_term_mat, feature_names, tfidf_vectorized

#Run NMF

In [97]:
def run_nmf(document_term_mat, num_components):
    nmf = NMF(n_components=num_components)
    W_sklearn = nmf.fit_transform(document_term_mat)
    H_sklearn = nmf.components_
    
    return nmf, W_sklearn, H_sklearn

In [80]:
def reconst_mse(target, left, right):
    '''An evaluation metric for NMF. Reconstructs the W and H metric.
    Compares them to the V matrix (the document term matrix).
    Returns the mean squared error, and evaluttion of how well the 
    W and H matrix approximate the V matrix.'''
    return (np.array(target - left.dot(right))**2).mean()

In [130]:
def describe_nmf_results(document_term_mat, W, H, feature_names, n_top_words = 30 ):
    """Uses the H matrix to determine the top words 
    in each uncovered latent feature"""
    #print("Reconstruction error: %f") %(reconst_mse(document_term_mat, W, H))
    for topic_num, topic in enumerate(H):
        print("\nTopic %d:" % topic_num)
        print(" ".join([feature_names[i] \
                for i in topic.argsort()[:-n_top_words - 1:-1]]))
     


In [203]:
nmf, W_sklearn, H_sklearn = run_nmf(document_term_mat, 10)
describe_nmf_results(document_term_mat,W_sklearn, H_sklearn,feature_names,n_top_words = 30,  )


Topic 0:
street sidewalk cleaning request http opened st iphone android garbage open311 trash pickup completed description 98 10 dumped leavenworth mattress folsom chair furniture comple blocking waste avenue lombard large 3rd

Topic 1:
https u2026 sf sanfrancisco ufe0f just day love today photo happy new u2764 amp good like best night mission coffee time california beach great tonight posted art morning life birthday

Topic 2:
st graffiti request opened intersection http iphone amp unattended neglected postings illegal abatement report neighborhood android left blighted fulton box signal neighborhoods church pole sign buchanan steiner fell mission anza

Topic 3:
hiring job careerarc jobs veterans hospitality http sanfrancisco retail ca starbucks latest click barista opening apply recommend fit shift work want great supervisor california sales center health financial manager nursing

Topic 4:
abandoned vehicles dpt gone arrival closed http request 15 07 opened iphone case st parked 08

0    1749
1     605
3     390
2     291
7     240
6     147
4     141
5     102
8      95
9      35
dtype: int64

In [ ]:
#15 clusters! 

In [327]:
nmf15, W_sklearn15, H_sklearn15 = run_nmf(document_term_mat, 15)
describe_nmf_results(document_term_mat,W_sklearn15, H_sklearn15,feature_names,n_top_words = 30,  )
#get the top cluster location for each geoid from the W matrix
geoid_cluster_labels15 = W_sklearn15.argsort( )[::-1][:, 0]
dftxt15clust = dftxt
dftxt15clust['cluster'] = geoid_cluster_labels15
dftxt15clust.cluster.value_counts()


Topic 0:
street sidewalk cleaning request opened http st iphone android garbage trash pickup open311 completed 98 description 10 dumped leavenworth mattress folsom chair furniture comple 3rd waste lombard blocking avenue shotwell

Topic 1:
san francisco ca https cafe restaurant market sf heights bar district muni california chinatown coffee food calif bay 24hourfitness event center stop house time station metro soma city neighborhoods amp

Topic 2:
st intersection unattended iphone amp neglected illegal postings request opened neighborhood http graffiti left blighted fulton anza church neighborhoods octavia divisadero page steiner fell 14th webster fillmore buchanan neigh neig

Topic 3:
closed case resolved completed st request http graffiti intersection amp transferred mission 15 pickup ses containers market process issues duplicate cleaned 18th meter streetlight sewer flooding psa maximo 17th 10

Topic 4:
hiring job careerarc veterans jobs hospitality sanfrancisco retail http ca sta

0     2229
1      697
2      308
4      270
5      122
3       92
6       31
8       18
7       14
9        5
12       4
11       3
14       2
dtype: int64

##What about with.... 20?

In [100]:
nmf20, W_sklearn20, H_sklearn20 =run_nmf(document_term_mat, 20)
describe_nmf_results(document_term_mat,W_sklearn20, H_sklearn20, feature_names )


Reconstruction error: 0.000128

Topic 0:
street sidewalk cleaning request opened http st iphone android pickup completed open311 trash 98 description 10 leavenworth mattress dumped folsom chair comple furniture waste lombard large 3rd shotwell avenue debris

Topic 1:
san francisco ca https cafe restaurant market sf bar california muni coffee calif food chinatown 24hourfitness event center stop time station soma district metro neighborhoods house bay amp city island

Topic 2:
graffiti abatement report st opened request android http signal box iphone intersection pole van ness shelter amp 3rd coach stockton bus division washington sign market kirkland line gough corner folsom

Topic 3:
closed case resolved completed st request http graffiti intersection amp transferred tree 15 ses maintenance pickup market process issues duplicate 18th streetlight sewer flooding cleaned meter maximo psa property 10

Topic 4:
hiring job careerarc veterans jobs hospitality sanfrancisco retail http ca starb

In [402]:
#get the top cluster location for each geoid from the W matrix
geoid_cluster_labels20 = W_sklearn20.argsort( )[::-1][:, 0]
df_20 = dftxt.copy()
df_20['cluster'] = geoid_cluster_labels20
df_20.cluster.value_counts()

0     2259
9     1109
19      88
17      74
1       74
16      65
2       44
8       27
15      14
4       13
13       7
5        7
3        6
7        4
11       1
18       1
14       1
6        1
dtype: int64

##What about 6?
###Answer: Six is too few. There are no subclusters.

In [205]:
mf6, W_sklearn6, H_sklearn6 =run_nmf(document_term_mat, 6)
describe_nmf_results(document_term_mat,W_sklearn6, H_sklearn6, feature_names )




Topic 0:
street sidewalk cleaning request opened http ave iphone st android garbage completed intersection open311 pickup trash amp case description 98 10 closed geary blvd dumped bruno mattress oakdale blocking leavenworth

Topic 1:
https san francisco u2026 ca sf sanfrancisco http just ufe0f day love park today photo coffee california time new cafe amp restaurant good happy u2764 golden night city bar like

Topic 2:
u0627 u0644 u064a u0648 u0645 u0646 u062a u0647 u0631 u0628 u0639 u062f u0643 u0633 u0642 u0641 u0634 u062d u0635 u0623 u0629 u062c u062e u0630 u0001f602 u0637 u0632 u061f u063a ness

Topic 3:
hiring job careerarc jobs veterans http hospitality ca sanfrancisco retail starbucks latest click barista opening san francisco fit apply recommend shift work want great supervisor financial center sales california manager

Topic 4:
closed case resolved st completed request http graffiti intersection abandoned vehicles amp 15 transferred tree mission maintenance containers park rep

#Test Two: Can it cluster tweets!

In [104]:
alltwt_doct = df.text.tolist()

In [134]:
docmatrix_alltwts20, featurenames_alltwt20, tfidfvectorized_alltwt20 = create_tfidf(alltwt_doct, 10000)
nmf_altwt20, W_altwt20, H_alltwt20 = run_nmf(docmatrix_alltwts20, 20)
   

In [137]:
describe_nmf_results(docmatrix_alltwts20, W_altwt20, H_alltwt20, featurenames_alltwt20 )



Topic 0:
closed case resolved completed st request http graffiti intersection amp transferred tree ses 15 maintenance market process pickup duplicate issues 18th streetlight sewer cleaned flooding meter property 17th maximo sr

Topic 1:
san francisco ca https cafe restaurant market muni bar sf california chinatown 24hourfitness coffee metro food calif bay center station district time stop neighborhoods house amp philzcoffee island body bank

Topic 2:
st unattended intersection iphone neglected amp illegal postings opened request neighborhood http graffiti left blighted fulton anza church neighborhoods divisadero octavia page fell steiner webster buchanan 14th fillmore neig alameda

Topic 3:
street sidewalk cleaning request opened http st iphone android completed pickup trash open311 98 10 description leavenworth mattress dumped folsom chair furniture waste lombard shotwell 3rd comple large 1098 newhall

Topic 4:
hiring job careerarc veterans jobs hospitality sanfrancisco retail ca htt

m

###Now Turn These documents into Maps

In [379]:
#get the top cluster location for each geoid from the W matrix
geoid_cluster_labels15 = W_sklearn15.argsort( )[::-1][:, 0]
dftxt15clust = dftxt.copy()
dftxt15clust['cluster'] = geoid_cluster_labels15
dftxt15clust.cluster.value_counts()

0     2229
1      697
2      308
4      270
5      122
3       92
6       31
8       18
7       14
9        5
12       4
11       3
14       2
dtype: int64

In [377]:
#get the top cluster location for each geoid from the W matrix
geoid_cluster_labels = W_sklearn.argsort( )[::-1][:, 0]
dftxt10clust = dftxt.copy()
dftxt10clust['cluster'] = geoid_cluster_labels

#The value count tells you how many geoids belong to each cluster. 
dftxt10clust.cluster.value_counts()



0    1749
1     605
3     390
2     291
7     240
6     147
4     141
5     102
8      95
9      35
dtype: int64

In [378]:
dftxt15clust.cluster.value_counts()

0     2229
1      697
2      308
4      270
5      122
3       92
6       31
8       18
7       14
9        5
12       4
11       3
14       2
dtype: int64

####Retrieve the geo files

In [380]:
geodf = pd.read_csv('data/intermediate_data/sf_only_sql_shapes.csv')
#format the dataframe
geodf['geoid10'] = geodf.geoid10.astype('str')
geodf.drop('Unnamed: 0', axis = 1, inplace = True)


####Merge the dataframes of labeled clusters with the shape files

In [381]:
#ned to format the geoid column for merging
dftxt10clust['geoid10'] =dftxt10clust['geoid10'].apply(lambda x: x[1:])
dftxt15clust['geoid10'] =dftxt15clust['geoid10'].apply(lambda x: x[1:])

In [383]:

df10clust = pd.merge(geodf, dftxt10clust, on='geoid10', how='outer')
df15clust = pd.merge(geodf, dftxt15clust, on='geoid10', how='outer')

In [384]:
#drop shape files missing tweets and clusters 
df15clust.dropna(inplace = True)
df10clust.dropna(inplace = True)

In [399]:
#make a dictionary to code each cluster as belonging to a different 'feature' group 
nmf_10dict = {0: 'city', 1: 'buisness_a', 2:'city', 3: 'hire', 4: 'city', 5: 'city', 6: 'buisness_b', 7: 'city', 8: 'city', 9: 'park_a'}
nmf_15dict = {0: 'city', 1: 'buisness_a', 2:'city', 3: 'city', 4: 'hire', 5: 'city', 6: 'city', 7: 'city', 8: 'city', 9: 'city', 10: 'city', 11:'park_a', 12: 'city', 13: 'park_b', 14: 'city'}

nmf_20dict = {0: 'city', 1: 'buisness_a', 2:'city', 3: 'city', 4: 'hire', 5: 'city', 6: 'city', 7: 'park_a', 8: 'city', 9: 'city', 10: 'city', 11:'park_b', 12: 'city', 13: 'city', 14: 'city', 15:'city', 16: 'city', 17: 'city', 18:'buisness_b', 19:'park_c' }

In [386]:
#initialize a dictionary for color mapping. 
#This maps each label to a different color, while preserving unique colors for each cluster.
color_dict = {
'park_a' :['#A2CD5A'],
'park_b' :['#BCEE68'],       
'park_c' :['#CAFF70'],
'buisness_a':['#E68A00'],
'buisness_b':['#FFAD33'],
'hire': ['#70B8FF'],
'city': ['#A3C2C2']}

In [403]:
df_20.head()

geoid10                                               text  cluster
0  060750101001000  [u'Acabo', u'de', u'publicar', u'una', u'foto'...        0
1  060750101001001  [u'a', u"gentleman's", u'quarters', u'at', u'#...        0
2  060750101001002  [u'Remember', u'why', u'you', u'started', u'ht...        0
3  060750101001004  [u'My', u'last', u'#SFO', u'post', u'\U0001f62...        0
4  060750101001005  [u'I', u'want', u'one', u'for', u'my', u'birth...        9

####Create geojason file

In [388]:
#assign a label to each cluster in the dataframe
df10clust['label'] = df10clust["cluster"].apply(lambda x:nmf_10dict[int(float(x))])
#assign a color to each cluster  
df10clust['hexcolor']=df10clust['label'].apply(lambda x : color_dict[x][0])

#assign a label to each cluster in the dataframe
df15clust['label'] = df15clust["cluster"].apply(lambda x:nmf_15dict[int(float(x))])
#assign a color to each cluster  
df15clust['hexcolor']=df15clust['label'].apply(lambda x : color_dict[x][0])

In [404]:
df_20['geoid10'] =df_20['geoid10'].apply(lambda x: x[1:])
df_20 = pd.merge(geodf, df_20, on='geoid10', how='outer')
df_20.dropna(inplace = True)


#assign a label to each cluster in the dataframe
df_20['label'] = df_20["cluster"].apply(lambda x:nmf_20dict[int(float(x))])
#assign a color to each cluster  
df_20['hexcolor']=df_20['label'].apply(lambda x : color_dict[x][0])




In [405]:
df_20.to_pickle('data/intermediate_data/nmf/nm_20df.pkl')
dataframe_to_geojson(df_20, 'map_making/clusters/data/clustergeo_nmf20.json')

In [337]:
df15clust.head()

Empty DataFrame
Columns: [geometry, geoid10, text, cluster, label, hexcolor]
Index: []

In [389]:
def add_properties_geo(row):
    '''
    INPUT: a row of a datafram.
    OUTPUT: A geo_json with the properties cluster, label and hexcolor.'''
    
    #assign relevant columns as properties
    geoid = row['geoid10']
    hexcolor = row['hexcolor']
    cluster = row['cluster']
    label = row['label']
    
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),\
                "properties": {'geoid': geoid ,'label': label,\
                'cluster': cluster , 'hexcolor' : hexcolor}}
    return geo_json

def dataframe_to_geojson(df, outfilename):
    '''
    INPUT: dataframe with a count, geoid10, and list of tokens.
    OUTPUT: A json geojason file dumped where the argument 'outfilename' specifies. 
    '''
    #reassign numeric columns as strings for json formatting
    df['geoid10'] = df['geoid10'].astype('str')
    df["cluster"] = df['cluster'].astype('str')
    
    #call function add_properties_geo on every row of the dataframe to create a geo_json.
    list_to_export = []
    for idx, row in df.iterrows():
        list_to_export.append(add_properties_geo(row))
    
    #dump the geo_json to the output assigned by 'outfilename'
    with open(outfilename, 'w') as outfile:
        json.dump(list_to_export, outfile)

In [406]:
#save dataframes with all tokens for later analysis
df10clust.to_pickle('data/intermediate_data/nmf/nmf10_df.pkl')
df15clust.to_pickle('data/intermediate_data/nmf/nm_15df.pkl')

dataframe_to_geojson(df10clust, 'map_making/clusters/data/clustergeo_nmf10.json')
dataframe_to_geojson(df15clust, 'map_making/clusters/data/clustergeo_nmf15.json')

In [325]:
df15clust.cluster.unique()

array(['1.0', '0.0', '6.0', '5.0', '3.0', '2.0', '4.0', '7.0', '8.0', '9.0'], dtype=object)

#What are the Latent Features NMF Uncovers?:
    

### HIRING
####20 CRITERIA

Topic 4: 13
hiring job careerarc veterans jobs hospitality sanfrancisco retail http ca starbucks latest click barista opening recommend fit apply shift want work supervisor great california sales health center manager financial nursing

####15 CRITERIA
Topic 4: 270
hiring job careerarc veterans jobs hospitality sanfrancisco retail http ca starbucks latest click barista opening recommend fit apply shift want work supervisor great california sales health center manager financial nursing

####10 CRITERIA
Topic 3:390
hiring job careerarc jobs veterans hospitality http sanfrancisco retail ca starbucks latest click barista opening apply recommend fit shift work want great supervisor california sales center health financial manager nursing


###PARKS
####20 CRITERIA
Topic 7: 4
park golden gate requests bridge lake https presidio dolores outsidelands national psa golf request glen marina ggp garden outside lands sanfrancisco opened manager area u0001f309 bay goldengatebridge day iphone recreation

Topic 16: 65
beach ocean sunset district baker sf oceanbeach north view taraval south day lands end 47th sutro parks4all bridge picnic u0001f30a marina u0001f309 sanfrancisco weekend california u2728 goldengate https fun china

####15 CRITERIA
Topic 11: 3
park golden gate requests bridge https lake presidio dolores heights outsidelands national psa marina golf bay glen ggp garden sanfrancisco bernal lands request outside u0001f309 manager posted just sunset area

####10 CRITERIA
Topic 9: 35
park golden gate requests bridge lake https http presidio dolores heights outsidelands national psa bay request glen marina golf ggp bernal garden sanfrancisco outside lands just opened manager day sunset

###LOCAL BUISNESS
####20 CRITERIA
Topic 12: 
mission st district dolores meter cleaned https bay graffiti 24th 16th http sf booze taqueria night minna sanfrancisco scheduled amp request 19th valencia intersection pie eat morning brazil good capp

Topic 17: 74
just photo posted hill heights https u2014 drinking bernal potrero bay bridge house nob diamond ipa chinatown peaks twin like russian 21stamendment kite sf good pizza pacific sanfrancisco ale brewing

Topic 1: 74
san francisco ca https cafe restaurant market sf bar california muni coffee calif food chinatown 24hourfitness event center stop time station soma district metro neighborhoods house bay amp city island

####15 CRITERIA
Topic 1: 697
san francisco ca https cafe restaurant market sf heights bar district muni california chinatown coffee food calif bay 24hourfitness event center stop house time station metro soma city neighborhoods amp
 
####10 CRITERIA
 
Topic 6:147
san francisco ca https http cafe restaurant sf market heights muni bar district california chinatown stop coffee time calif food center bay event metro 24hourfitness station house soma city drinking

Topic 1:0605
https u2026 sf sanfrancisco ufe0f just day love today photo happy new u2764 amp good like best night mission coffee time california beach great tonight posted art morning life birthday
 
###ONE HAPPY PERSON
####20 CRITERIA

Topic 14:  1
https u2026 sf sanfrancisco love day today amp new happy coffee best night time good california tonight like great birthday art life thanks amazing morning city summer come bar don



####15 CRITERIA
Topic 13: -
https u2026 sf sanfrancisco just day love today amp happy photo new good coffee night mission best like california time great tonight posted beach birthday art life morning thanks amazing

####10 CRITERIA

###City Buiness     

####20 CRITERIA

Topic 0: 2259
street sidewalk cleaning request opened http st iphone android pickup completed open311 trash 98 description 10 leavenworth mattress dumped folsom chair comple furniture waste lombard large 3rd shotwell avenue debris

Topic 18: 1
parking blocked space sidewalk blocking traffic request opened http sign iphone residential cecelia car closed repair 80 open311 bayarea 16th lane case description pmlohzdgyh invalid accident st way 101 nb

Topic 3: 6
closed case resolved completed st request http graffiti intersection amp transferred tree 15 ses maintenance pickup market process issues duplicate 18th streetlight sewer flooding cleaned meter maximo psa property 10

Topic 19: 88
http tree maintenance sf drinking u2014 today request opened like new good way really u201d u201c open311 u2019s defects time potholes need iphone work streetlight repair public u0001f602 don sewer 

Topic 15:14
garbage containers android st request opened http left oakdale bin serviced gough outreach overflowing city trash cans 3rd public 20th iphone noe naples rosa loose resolved 14th francisco buchanan dumped

Topic 5: 7
abandoned vehicles dpt gone arrival closed request 15 http 07 opened iphone case st parked 08 vehicle duplicate 28 car dr week bernal 03 35th shore ford 17th leavenworth playa

Topic 11: 1
st intersection unattended amp iphone neglected illegal postings request opened neighborhood http graffiti left blighted fulton anza church neighborhoods divisadero octavia page fell steiner webster 14th buchanan fillmore neigh neig

Topic 2:44
graffiti abatement report st opened request android http signal box iphone intersection pole van ness shelter amp 3rd coach stockton bus division washington sign market kirkland line gough corner folsom

Topic 10:
blvd geary iphone intersection amp shore request monterey bay opened alemany arguello defects http potholes sign presidio graffiti repair 37th signal box streetlight 12th closed neighborh anza case webster clement

####15 CRITERIA
Topic 0: 229
street sidewalk cleaning request opened http st iphone android garbage trash pickup open311 completed 98 description 10 dumped leavenworth mattress folsom chair furniture comple 3rd waste lombard blocking avenue shotwell

Topic 2:308
st intersection unattended iphone amp neglected illegal postings request opened neighborhood http graffiti left blighted fulton anza church neighborhoods octavia divisadero page steiner fell 14th webster fillmore buchanan neigh neig

Topic 3: 92
closed case resolved completed st request http graffiti intersection amp transferred mission 15 pickup ses containers market process issues duplicate cleaned 18th meter streetlight sewer flooding psa maximo 17th 10

Topic 5: 122
abandoned vehicles dpt gone arrival closed request 15 http 07 opened iphone case st parked 08 vehicle duplicate 28 car bernal dr week 03 heights shore 35th ford 17th leavenworth

Topic 6: 31
ave intersection amp request bruno opened iphone case android closed http parking jose sidewalk van repair anza ness san ocean cabrillo oakdale potrero cleaning clement corner 45th 19th masonic lobos

Topic 7: 14
graffiti abatement report st request opened android http signal box iphone intersection mission pole van amp ness shelter 3rd bus coach stockton division market washington line sign kirkland gough dolores

Topic 9: 5
blvd geary iphone intersection amp shore request bay monterey opened alemany arguello defects http potholes graffiti sign presidio repair signal 37th closed box case 12th streetlight anza neighborh webster clement

Topic 10:
http parking space blocked drinking u2014 garbage containers traffic request sf opened repair new open311 like today good just way really sign u201d u201c u2019s blocking description defects photo time

Topic 14:2
tree maintenance request opened iphone http st trees way monterey dr topped public ses branch buchanan university broke lily letter scott 1021 open311 park winston page hitting electric diamond sent

####10 CRITERIA
Topic 8:95
ave intersection amp blvd request opened http geary iphone bruno android closed parking case repair anza van ness sidewalk graffiti jose sign cabrillo san clement ocean oakdale traffic potrero illegal

Topic 4:141
abandoned vehicles dpt gone arrival closed http request 15 07 opened iphone case st parked 08 vehicle duplicate car 28 dr bernal week ave 03 heights shore 35th ford open311

Topic 5: 102
closed case resolved completed request st http graffiti intersection amp transferred tree mission maintenance 15 containers ses pickup issues market process meter cleaned duplicate streetlight sewer 18th flooding garbage repair

Topic 2: 291
st graffiti request opened intersection http iphone amp unattended neglected postings illegal abatement report neighborhood android left blighted fulton box signal neighborhoods church pole sign buchanan steiner fell mission anza


###MISC

Topic 6: 1
ave intersection amp request bruno opened http case closed iphone android jose anza van ness repair san cabrillo potrero oakdale cleaning clement corner 19th 45th masonic sidewalk lobos silver ocean

###UNICODE
####20 CRITERIA
Topic 13: 7
u043e u0430 u0435 u043d u0442 u0438 u043a u0441 u0440 u0432 u043c u043b u0443 u043f u0434 u044b u0433 u044c u0431 u044f u0447 u0439 u0437 fillmore u044e castro u0448 u0436 https gt

Topic 8:27
ufe0f u2764 u2600 u263a u0001f60d u0001f64f home u0001f44c u0001f43e https u0001f602 u0001f3e1 love u0001f4af loving u26a1 u0001f44d u0001f3fb u2714 u0001f49b best u270c miss house u2615 u0001f618 periscope twin u2601 u0001f44f

Topic 9:1109
u0627 u0644 u064a u0648 u0645 u0646 u062a u0631 u0647 u0628 u0639 u062f u0643 u0633 u0642 u0641 u0634 u062d u0635 u0623 u0629 u062c u062e u0630 u0001f602 u0637 u0632 u061f u063a u06cc

####15 CRITERIA
Topic 8: 18
u0627 u0644 u064a u0648 u0645 u0646 u062a u0631 u0647 u0628 u0639 u062f u0643 u0633 u0642 u0641 u0634 u062d u0635 u0623 u0629 u062c u062e u0630 u0001f602 u0637 u0632 u061f u063a u06cc

Topic 12: 4
ufe0f u2764 u043e u0430 u0435 u043d u0442 u0438 u043a u0441 u2600 u0440 u043c u0432 u263a beach u0443 u043b u0001f64f https u0001f60d u043f home u0434 u0001f43e u0001f44c u0001f3e1 loving u26a1 u0001f4af

####10 CRITERIA

Topic 7: 240
u0627 u0644 u064a u0648 u0645 u0646 u062a u0631 u0647 u0628 u0639 u062f u0643 u0633 u0642 u0641 u0634 u062d u0635 u0623 u0629 u062c u062e u0630 u0001f602 u0637 u0632 u061f u063a ness

##conclusions: 
clustering by tweets themselves was not particularly more informative, the clusters look very similar.

As far as the number of latent features to draw out, six did not seem to seperate them into useful clusters.

With ten latent features, I start to get categories:

- <b>Parks</b>: tweets related to parks, oceans, outside
- <b>Local Buisnesses</b>: tweets related to coffee, breweries, ect
- <b>Hiring</b>: tweets related to open calls for jobs
- <b>Graffiti/Trafic</b>: tweets related to grafitti, parking violations, potholes. Less related to freetime activities and more to maintence of city.

For 10, 15, and 20 latent features, multiple geoids emerged from these subsections, with each of them parsing them slightly different. I think 15 is a nice balance.